In [1]:
import os
import sys
sys.path.append('../src')

from dotenv import load_dotenv
from models import Review
from db import DataBaseConnector
from processing import SteamReviewsDownloader
load_dotenv()

True

In [2]:
app_ids = {'cs2':730}
downloader = SteamReviewsDownloader(app_ids)

In [3]:
reviews = downloader.run()

20:51:15.598 | INFO    | prefect.engine - Created flow run 'humongous-bullfrog' for flow 'run'

20:51:15.599 | INFO    | prefect.engine - View at http://127.0.0.1:4200/runs/flow-run/4a181d7c-b979-4ad2-8b04-aa7636743c31

20:51:15.637 | INFO    | Task run 'download_reviews-87b' - Created task run 'download_reviews-87b' for task 'download_reviews'

20:51:15.684 | INFO    | Task run 'download_reviews-87b' - Finished in state Cached(type=COMPLETED)

20:51:15.717 | INFO    | Task run 'add_game_name-1a7' - Created task run 'add_game_name-1a7' for task 'add_game_name'

20:51:15.762 | INFO    | Task run 'add_game_name-1a7' - Finished in state Completed()

20:51:15.800 | INFO    | Flow run 'humongous-bullfrog' - Finished in state Completed()

In [4]:
reviews[0]

{'recommendationid': '172440169',
 'author': {'steamid': '76561198893790695',
  'num_games_owned': 0,
  'num_reviews': 1,
  'playtime_forever': 2862,
  'playtime_last_two_weeks': 0,
  'playtime_at_review': 2862,
  'deck_playtime_at_review': 941,
  'last_played': 1715537143},
 'language': 'english',
 'review': "terrible anti cheat people are getting false bans for high sensitivity/amd drivers/console commands while real cheaters are not getting banned\n- no new content for months\n- no new operation (it's been 3+ years)\n- tons of competitive/wingman/hostage maps missing from csgo\n- deleted achievements from csgo with 1 pointless one just make new ones\n- no danger zone\n- no team deatmatch\n- sub tick is garbage all we wanted was 128tick servers\n- spaghetti net code\n- no short mm\n- no flying scoutsman\n- replays are bugged\n- no overwatch\n- performance issues\n- peekers advantage\n- bad ranking system\n- no economy changes despite MR12\n- movement and shooting is worse compared to

In [6]:
all_reviews[0]

<Review(recommendationid=172440169, voted_up=terrible anti cheat people are getting false bans for high sensitivity/amd drivers/console commands while real cheaters are not getting banned
- no new content for months
- no new operation (it's been 3+ years)
- tons of competitive/wingman/hostage maps missing from csgo
- deleted achievements from csgo with 1 pointless one just make new ones
- no danger zone
- no team deatmatch
- sub tick is garbage all we wanted was 128tick servers
- spaghetti net code
- no short mm
- no flying scoutsman
- replays are bugged
- no overwatch
- performance issues
- peekers advantage
- bad ranking system
- no economy changes despite MR12
- movement and shooting is worse compared to csgo

All of this while they make hundreds of millions of dollars from cases/keys and yet are unable to make any fun new content unbelievable, and yet they waste their time with boring hero shooter deadlock that they will abandon within the first few months after the release just li

In [7]:
from sqlalchemy.dialects.postgresql import insert

all_reviews = []
for r in reviews:
    del r['author']

    all_reviews.append(Review(**r))

db_config = {
        "db_type": "postgresql",
        'host':'localhost',
        'port':5432,
        'database':'game_assistant',
        'user':'postgres',
        'password':'postgres',
    }
db = DataBaseConnector(**db_config)


with db.session_scope() as session:
    try:
        stmt = insert(Review).values(reviews)
        stmt = stmt.on_conflict_do_update(
            index_elements=['recommendationid'],
            set_={c.key: c for c in stmt.excluded if c.key != 'recommendationid'}
        )
        result = session.execute(stmt)
        saved_count = result.rowcount
        print(saved_count)
    except Exception as err:
        print(err)
        session.rollback()



100
